# 🌍 การจัดการข้อมูล Raster ด้วย Python
## Rasterio, Xarray, Rioxarray และ XEE

**ผู้สอน:** ดร.สิทธิ์ เจตน์มงคล | ภาควิชาภูมิศาสตร์ คณะศิลปศาสตร์ มหาวิทยาลัยธรรมศาสตร์

---

### สิ่งที่จะได้เรียนรู้

| หัวข้อ | เนื้อหา |
|--------|--------|
| **Part 1** | พื้นฐานข้อมูล Raster และการติดตั้ง Library |
| **Part 2** | Rasterio — โหลด อ่าน เขียน และวิเคราะห์ Raster |
| **Part 3** | Xarray — จัดการ Raster แบบ N-dimensional |
| **Part 4** | Rioxarray — สะพานเชื่อม Rasterio + Xarray |
| **Part 5** | XEE — เชื่อมต่อ Google Earth Engine กับ Xarray |
| **Part 6** | Workshop รวม — ประยุกต์ใช้งานจริง |

---
# Part 1: พื้นฐานข้อมูล Raster และการติดตั้ง

## 1.1 ข้อมูล Raster คืออะไร?

ข้อมูล Raster คือข้อมูลเชิงพื้นที่ที่จัดเก็บในรูปแบบ **กริด (Grid)** ของ **พิกเซล (Pixel)** แต่ละพิกเซลเก็บค่าตัวเลขที่แสดงถึงค่าบางอย่าง เช่น ค่าการสะท้อนแสง อุณหภูมิ ความสูง ฯลฯ

### องค์ประกอบสำคัญของข้อมูล Raster

| องค์ประกอบ | คำอธิบาย | ตัวอย่าง |
|-----------|----------|----------|
| **Pixel/Cell** | หน่วยย่อยที่สุดของ Raster | พิกเซลขนาด 30m × 30m |
| **Band** | ชั้นข้อมูลแต่ละชั้น | Band 1 = Red, Band 2 = Green |
| **Resolution** | ขนาดของพิกเซลบนพื้นโลก | 10m (Sentinel-2), 30m (Landsat) |
| **CRS** | ระบบพิกัดอ้างอิง | EPSG:4326 (WGS84), EPSG:32647 (UTM47N) |
| **Extent/Bounds** | ขอบเขตพื้นที่ | (xmin, ymin, xmax, ymax) |
| **NoData** | ค่าที่ใช้แทนพื้นที่ไม่มีข้อมูล | -9999, NaN |
| **Transform** | การแปลงพิกัดจาก pixel → map | Affine transformation matrix |

### รูปแบบไฟล์ Raster ที่พบบ่อย

| Format | นามสกุล | จุดเด่น |
|--------|---------|--------|
| **GeoTIFF** | `.tif` | มาตรฐานสากล รองรับ CRS |
| **Cloud Optimized GeoTIFF (COG)** | `.tif` | เหมาะกับ cloud, อ่านได้บางส่วน |
| **NetCDF** | `.nc` | เก็บข้อมูลหลายมิติ (เวลา, ความสูง) |
| **HDF5** | `.hdf`, `.h5` | NASA ใช้มาก (MODIS, VIIRS) |
| **JPEG2000** | `.jp2` | Sentinel-2 ใช้ |
| **Zarr** | `.zarr` | สำหรับ cloud-native, chunked |

## 1.2 ภาพรวม Library ที่ใช้

```
┌─────────────────────────────────────────────────────────────┐
│                    Python Raster Ecosystem                   │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐               │
│  │ Rasterio │    │  Xarray  │    │   XEE    │               │
│  │ (I/O)    │◄──►│ (N-dim)  │◄──►│ (GEE)   │               │
│  └────┬─────┘    └────┬─────┘    └──────────┘               │
│       │               │                                      │
│       └───────┬───────┘                                      │
│               │                                              │
│        ┌──────┴──────┐                                       │
│        │  Rioxarray  │  (สะพานเชื่อม)                        │
│        └──────┬──────┘                                       │
│               │                                              │
│        ┌──────┴──────┐                                       │
│        │    GDAL     │  (Engine หลัก)                        │
│        └─────────────┘                                       │
└─────────────────────────────────────────────────────────────┘
```

| Library | หน้าที่หลัก | เมื่อไหร่ควรใช้ |
|---------|-----------|---------------|
| **Rasterio** | อ่าน/เขียน/แปลงไฟล์ Raster | งาน I/O ทั่วไป, ประมวลผล band-by-band |
| **Xarray** | จัดการข้อมูลหลายมิติ (x, y, time, band) | Time-series, NetCDF, วิเคราะห์หลายมิติ |
| **Rioxarray** | เพิ่ม CRS/Transform ให้ Xarray | เมื่อใช้ Xarray แต่ต้องการ geo-awareness |
| **XEE** | ดึงข้อมูลจาก Google Earth Engine → Xarray | เมื่อต้องการข้อมูล GEE มาวิเคราะห์ใน Python |

## 1.3 การติดตั้ง Library

In [ ]:
# ติดตั้ง Library ที่จำเป็นทั้งหมด
# รันครั้งเดียว แล้ว restart kernel

!pip install rasterio xarray rioxarray netcdf4 h5netcdf matplotlib
!pip install xee earthengine-api
!pip install dask[complete]  # สำหรับ lazy loading ข้อมูลขนาดใหญ่

In [ ]:
# ตรวจสอบเวอร์ชัน
import rasterio
import xarray as xr
import rioxarray
import numpy as np
import matplotlib.pyplot as plt

print(f"Rasterio  : {rasterio.__version__}")
print(f"Xarray    : {xr.__version__}")
print(f"NumPy     : {np.__version__}")

## 1.4 เตรียมข้อมูลตัวอย่าง

เราจะสร้างข้อมูล Raster ตัวอย่างเพื่อใช้ในการเรียนรู้ และยังแสดงวิธีดาวน์โหลดข้อมูลจริงด้วย

In [ ]:
# สร้างข้อมูล Raster ตัวอย่าง (DEM จำลอง)
import rasterio
from rasterio.transform import from_bounds
import numpy as np
import os

# สร้างโฟลเดอร์เก็บข้อมูล
os.makedirs('data', exist_ok=True)

# สร้าง DEM จำลองของพื้นที่ภาคเหนือของไทย
# ขอบเขต: ลองจิจูด 98-100°E, ละติจูด 18-20°N
nrows, ncols = 200, 200
xmin, ymin, xmax, ymax = 98.0, 18.0, 100.0, 20.0

# สร้างพื้นผิวจำลอง (ใช้ sin/cos สร้างภูเขาจำลอง)
x = np.linspace(0, 4*np.pi, ncols)
y = np.linspace(0, 4*np.pi, nrows)
X, Y = np.meshgrid(x, y)
elevation = 300 + 200*np.sin(X)*np.cos(Y) + 100*np.sin(2*X) + np.random.normal(0, 10, (nrows, ncols))
elevation = elevation.astype(np.float32)

# กำหนด Transform (affine transformation)
transform = from_bounds(xmin, ymin, xmax, ymax, ncols, nrows)

# เขียนไฟล์ GeoTIFF
with rasterio.open(
    'data/dem_sample.tif',
    'w',
    driver='GTiff',
    height=nrows,
    width=ncols,
    count=1,           # จำนวน band
    dtype='float32',
    crs='EPSG:4326',   # WGS84
    transform=transform,
    nodata=-9999
) as dst:
    dst.write(elevation, 1)  # เขียน band 1

print("✅ สร้างไฟล์ data/dem_sample.tif เรียบร้อย")
print(f"   ขนาด: {ncols} x {nrows} pixels")
print(f"   ขอบเขต: {xmin}°E - {xmax}°E, {ymin}°N - {ymax}°N")

In [ ]:
# สร้างข้อมูล Multispectral จำลอง (เหมือน Landsat 4 bands)
bands = 4  # Blue, Green, Red, NIR
nrows, ncols = 200, 200

np.random.seed(42)

# จำลอง spectral response
# สร้างพื้นที่ป่า (สูง NIR, ต่ำ Red) และพื้นที่เมือง (สูง Red, ต่ำ NIR)
forest_mask = (X > 2*np.pi).astype(float)  # ครึ่งขวาเป็นป่า

multispectral = np.zeros((bands, nrows, ncols), dtype=np.float32)
# Blue band
multispectral[0] = 0.05 + 0.03*forest_mask + np.random.normal(0, 0.01, (nrows, ncols))
# Green band
multispectral[1] = 0.06 + 0.05*forest_mask + np.random.normal(0, 0.01, (nrows, ncols))
# Red band
multispectral[2] = 0.10 - 0.05*forest_mask + np.random.normal(0, 0.01, (nrows, ncols))
# NIR band
multispectral[3] = 0.15 + 0.25*forest_mask + np.random.normal(0, 0.02, (nrows, ncols))

multispectral = np.clip(multispectral, 0, 1)  # จำกัดค่า 0-1

transform = from_bounds(98.0, 18.0, 100.0, 20.0, ncols, nrows)

with rasterio.open(
    'data/multispectral_sample.tif',
    'w',
    driver='GTiff',
    height=nrows,
    width=ncols,
    count=bands,
    dtype='float32',
    crs='EPSG:4326',
    transform=transform,
    nodata=-9999
) as dst:
    dst.write(multispectral)
    # ตั้งชื่อ band
    dst.update_tags(1, name='Blue')
    dst.update_tags(2, name='Green')
    dst.update_tags(3, name='Red')
    dst.update_tags(4, name='NIR')

print("✅ สร้างไฟล์ data/multispectral_sample.tif เรียบร้อย")
print(f"   ขนาด: {ncols} x {nrows} pixels, {bands} bands")

In [ ]:
# สร้างข้อมูล Time-series จำลอง (อุณหภูมิรายเดือน) เป็น NetCDF
import xarray as xr
import pandas as pd

# สร้างข้อมูลอุณหภูมิรายเดือนของภาคเหนือ (12 เดือน)
times = pd.date_range('2024-01-01', periods=12, freq='MS')
lats = np.linspace(20.0, 18.0, 50)
lons = np.linspace(98.0, 100.0, 50)

# อุณหภูมิจำลอง (มีรูปแบบตามฤดูกาล)
temp_data = np.zeros((12, 50, 50), dtype=np.float32)
for m in range(12):
    # อุณหภูมิฐาน + seasonal pattern + spatial variation
    base_temp = 25 + 8*np.sin(2*np.pi*(m-3)/12)  # ร้อนสุดเดือน เม.ย.
    lat_effect = np.linspace(2, -2, 50).reshape(-1, 1)  # เหนือเย็นกว่า
    temp_data[m] = base_temp + lat_effect + np.random.normal(0, 1, (50, 50))

# สร้าง Dataset
ds = xr.Dataset(
    {
        'temperature': (['time', 'latitude', 'longitude'], temp_data, 
                        {'units': '°C', 'long_name': 'Surface Temperature'}),
    },
    coords={
        'time': times,
        'latitude': lats,
        'longitude': lons,
    },
    attrs={
        'title': 'Monthly Temperature - Northern Thailand (Simulated)',
        'source': 'Simulated data for teaching',
        'crs': 'EPSG:4326'
    }
)

ds.to_netcdf('data/temperature_monthly.nc')
print("✅ สร้างไฟล์ data/temperature_monthly.nc เรียบร้อย")
print(ds)

---
# Part 2: Rasterio — โหลด อ่าน เขียน และวิเคราะห์ Raster

**Rasterio** เป็น Library หลักสำหรับอ่าน/เขียนไฟล์ Raster ใน Python สร้างบน GDAL แต่ใช้งานง่ายกว่ามาก

## 2.1 เปิดและสำรวจไฟล์ Raster

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

# เปิดไฟล์ (ยังไม่โหลดข้อมูลเข้า memory)
src = rasterio.open('data/dem_sample.tif')

# ดู metadata ทั้งหมด
print("=" * 50)
print("📋 METADATA ของไฟล์ DEM")
print("=" * 50)
print(f"Driver     : {src.driver}")        # รูปแบบไฟล์
print(f"Width      : {src.width} pixels")   # จำนวนคอลัมน์
print(f"Height     : {src.height} pixels")  # จำนวนแถว
print(f"Count      : {src.count} band(s)")  # จำนวน band
print(f"CRS        : {src.crs}")            # ระบบพิกัด
print(f"Bounds     : {src.bounds}")          # ขอบเขต
print(f"Resolution : {src.res}")             # ความละเอียด (x, y)
print(f"Dtype      : {src.dtypes}")          # ชนิดข้อมูล
print(f"NoData     : {src.nodata}")          # ค่า NoData
print(f"Transform  :\n{src.transform}")      # Affine transform

### ทำความเข้าใจ Affine Transform

Affine Transform เป็น matrix 3×3 ที่ใช้แปลงพิกัด **pixel (col, row)** → **map (x, y)**

```
| a  b  c |     a = pixel width (x-resolution)
| d  e  f |     b = row rotation (ปกติ = 0)
| 0  0  1 |     c = x ของ upper-left corner
                 d = column rotation (ปกติ = 0)
                 e = pixel height (ค่าลบ เพราะ row นับจากบนลงล่าง)
                 f = y ของ upper-left corner
```

สูตร: `x = a * col + b * row + c`  
สูตร: `y = d * col + e * row + f`

In [ ]:
# ตัวอย่างการแปลงพิกัด
transform = src.transform

# แปลง pixel → map coordinates
# pixel (0, 0) = มุมซ้ายบน
x, y = transform * (0, 0)
print(f"Pixel (0,0)   → Map ({x:.4f}, {y:.4f})")

# pixel (100, 100) = กลางภาพ
x, y = transform * (100, 100)
print(f"Pixel (100,100) → Map ({x:.4f}, {y:.4f})")

# แปลง map → pixel coordinates (inverse)
col, row = ~transform * (99.0, 19.0)
print(f"\nMap (99.0, 19.0) → Pixel ({col:.1f}, {row:.1f})")

## 2.2 การอ่านข้อมูล Raster

In [ ]:
# วิธีที่ 1: อ่านทั้ง band (ใช้กับไฟล์ขนาดเล็ก)
with rasterio.open('data/dem_sample.tif') as src:
    dem = src.read(1)  # อ่าน band 1 → numpy array 2D
    profile = src.profile  # เก็บ metadata ไว้ใช้ทีหลัง

print(f"Shape: {dem.shape}")
print(f"Dtype: {dem.dtype}")
print(f"Min: {dem.min():.1f}, Max: {dem.max():.1f}, Mean: {dem.mean():.1f}")

In [ ]:
# วิธีที่ 2: อ่านหลาย band พร้อมกัน
with rasterio.open('data/multispectral_sample.tif') as src:
    # อ่านทุก band → (bands, rows, cols)
    all_bands = src.read()  
    print(f"อ่านทุก band: {all_bands.shape}")
    
    # อ่านเฉพาะ band ที่ต้องการ
    red_nir = src.read([3, 4])  # เฉพาะ Red และ NIR
    print(f"อ่าน Red+NIR: {red_nir.shape}")

In [ ]:
# วิธีที่ 3: อ่านเฉพาะบางส่วน (Window) — สำคัญมากสำหรับไฟล์ใหญ่!
from rasterio.windows import Window

with rasterio.open('data/dem_sample.tif') as src:
    # Window(col_off, row_off, width, height)
    window = Window(50, 50, 100, 100)  # อ่านเฉพาะส่วนกลาง 100×100
    
    subset = src.read(1, window=window)
    print(f"อ่านเฉพาะ window: {subset.shape}")
    
    # ดู bounds ของ window
    window_transform = src.window_transform(window)
    window_bounds = rasterio.windows.bounds(window, src.transform)
    print(f"Bounds ของ window: {window_bounds}")

In [ ]:
# วิธีที่ 4: อ่านโดยกำหนดขอบเขตทางภูมิศาสตร์
from rasterio.windows import from_bounds

with rasterio.open('data/dem_sample.tif') as src:
    # อ่านเฉพาะพื้นที่ lon 98.5-99.5, lat 18.5-19.5
    window = from_bounds(98.5, 18.5, 99.5, 19.5, src.transform)
    
    subset_geo = src.read(1, window=window)
    print(f"อ่านจาก bounds: {subset_geo.shape}")

In [ ]:
# วิธีที่ 5: อ่านพร้อม Resample (เปลี่ยนความละเอียด)
from rasterio.enums import Resampling

with rasterio.open('data/dem_sample.tif') as src:
    # อ่านโดยย่อขนาดลง 4 เท่า
    out_shape = (1, src.height // 4, src.width // 4)
    
    resampled = src.read(
        out_shape=out_shape,
        resampling=Resampling.bilinear  # วิธี interpolation
    )
    print(f"ขนาดเดิม: {src.height}×{src.width}")
    print(f"Resample: {resampled.shape[1]}×{resampled.shape[2]}")
    print(f"\nResampling methods ที่ใช้บ่อย:")
    print(f"  nearest  - เร็วสุด, เหมาะกับ categorical data")
    print(f"  bilinear - เหมาะกับ continuous data (DEM, temperature)")
    print(f"  cubic    - เรียบสุด แต่ช้ากว่า")

## 2.3 การแสดงผล Raster

In [ ]:
# แสดงผล DEM
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# วิธีที่ 1: ใช้ rasterio.plot.show (สะดวก มี geo-coordinates อัตโนมัติ)
with rasterio.open('data/dem_sample.tif') as src:
    show(src, ax=axes[0], cmap='terrain', title='rasterio.plot.show')

# วิธีที่ 2: ใช้ matplotlib โดยตรง (ควบคุมได้มากกว่า)
im = axes[1].imshow(dem, cmap='terrain', 
                     extent=[98, 100, 18, 20])  # กำหนด extent เอง
axes[1].set_title('matplotlib imshow')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
plt.colorbar(im, ax=axes[1], label='Elevation (m)', shrink=0.8)

# วิธีที่ 3: Hillshade effect
from matplotlib.colors import LightSource
ls = LightSource(azdeg=315, altdeg=45)
hillshade = ls.hillshade(dem, vert_exag=2)
axes[2].imshow(hillshade, cmap='gray', extent=[98, 100, 18, 20])
axes[2].set_title('Hillshade')
axes[2].set_xlabel('Longitude')

plt.tight_layout()
plt.show()

In [ ]:
# แสดงผลภาพ Multispectral
with rasterio.open('data/multispectral_sample.tif') as src:
    bands = src.read()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# True Color (RGB)
rgb = np.stack([bands[2], bands[1], bands[0]], axis=-1)  # Red, Green, Blue
rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min())  # Normalize to 0-1
axes[0].imshow(rgb_norm, extent=[98, 100, 18, 20])
axes[0].set_title('True Color (RGB)')

# False Color (NIR-R-G)
fcc = np.stack([bands[3], bands[2], bands[1]], axis=-1)
fcc_norm = (fcc - fcc.min()) / (fcc.max() - fcc.min())
axes[1].imshow(fcc_norm, extent=[98, 100, 18, 20])
axes[1].set_title('False Color (NIR-R-G)')

# NDVI
red = bands[2].astype(float)
nir = bands[3].astype(float)
ndvi = (nir - red) / (nir + red + 1e-10)
im = axes[2].imshow(ndvi, cmap='RdYlGn', vmin=-0.2, vmax=0.8,
                     extent=[98, 100, 18, 20])
axes[2].set_title('NDVI')
plt.colorbar(im, ax=axes[2], label='NDVI', shrink=0.8)

for ax in axes:
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

plt.tight_layout()
plt.show()

## 2.4 การเขียนไฟล์ Raster

In [ ]:
# เขียน NDVI เป็นไฟล์ GeoTIFF ใหม่
with rasterio.open('data/multispectral_sample.tif') as src:
    # คัดลอก profile แล้วแก้ไข
    out_profile = src.profile.copy()
    out_profile.update(
        count=1,           # NDVI มีแค่ 1 band
        dtype='float32',
        nodata=-9999,
        compress='lzw'     # บีบอัดไฟล์
    )
    
    # อ่านข้อมูลและคำนวณ NDVI
    red = src.read(3).astype(float)
    nir = src.read(4).astype(float)
    ndvi = (nir - red) / (nir + red + 1e-10)
    ndvi = ndvi.astype(np.float32)

# เขียนไฟล์
with rasterio.open('data/ndvi_output.tif', 'w', **out_profile) as dst:
    dst.write(ndvi, 1)
    dst.update_tags(1, name='NDVI')  # เพิ่ม metadata
    dst.update_tags(description='NDVI calculated from simulated multispectral data')

print("✅ เขียนไฟล์ data/ndvi_output.tif เรียบร้อย")

# ตรวจสอบ
with rasterio.open('data/ndvi_output.tif') as check:
    print(f"   Compress: {check.compression}")
    print(f"   Band tags: {check.tags(1)}")

In [ ]:
# เขียนเป็น Cloud Optimized GeoTIFF (COG)
# COG มีโครงสร้างพิเศษที่ช่วยให้อ่านข้อมูลบางส่วนผ่าน HTTP ได้เร็ว

with rasterio.open('data/multispectral_sample.tif') as src:
    out_profile = src.profile.copy()
    out_profile.update(
        driver='GTiff',
        compress='deflate',
        tiled=True,           # ⭐ จำเป็นสำหรับ COG
        blockxsize=256,       # ⭐ ขนาด tile
        blockysize=256,
    )
    
    with rasterio.open('data/multispectral_cog.tif', 'w', **out_profile) as dst:
        dst.write(src.read())
        
        # สร้าง overviews (ภาพย่อสำหรับ zoom levels ต่างๆ)
        dst.build_overviews([2, 4, 8], Resampling.average)
        dst.update_tags(ns='rio_overview', resampling='average')

print("✅ เขียนไฟล์ COG เรียบร้อย")
print("💡 COG เหมาะกับการเก็บบน Cloud (S3, GCS) เพราะอ่านได้ทีละส่วน")

## 2.5 การ Reproject (เปลี่ยนระบบพิกัด)

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:32647'  # UTM Zone 47N (เหมาะกับภาคเหนือไทย)

with rasterio.open('data/dem_sample.tif') as src:
    # คำนวณ transform ใหม่สำหรับ CRS ปลายทาง
    transform, width, height = calculate_default_transform(
        src.crs,       # CRS ต้นทาง
        dst_crs,       # CRS ปลายทาง
        src.width,     # ความกว้างเดิม
        src.height,    # ความสูงเดิม
        *src.bounds    # ขอบเขตเดิม
    )
    
    # เตรียม profile สำหรับไฟล์ใหม่
    out_profile = src.profile.copy()
    out_profile.update(
        crs=dst_crs,
        transform=transform,
        width=width,
        height=height
    )
    
    with rasterio.open('data/dem_utm47n.tif', 'w', **out_profile) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.bilinear
            )

# ตรวจสอบผลลัพธ์
with rasterio.open('data/dem_utm47n.tif') as check:
    print(f"✅ Reproject เรียบร้อย!")
    print(f"   CRS เดิม : EPSG:4326 (หน่วย: องศา)")
    print(f"   CRS ใหม่ : {check.crs} (หน่วย: เมตร)")
    print(f"   Resolution: {check.res[0]:.1f} x {check.res[1]:.1f} meters")
    print(f"   Size: {check.width} x {check.height}")

## 2.6 การ Clip (ตัดภาพด้วย Shapefile)

In [ ]:
from rasterio.mask import mask
from shapely.geometry import box, mapping
import json

# สร้าง polygon สำหรับ clip (จำลองขอบเขตจังหวัด)
clip_polygon = box(98.5, 18.5, 99.5, 19.5)  # สี่เหลี่ยม

# แปลงเป็น GeoJSON format ที่ rasterio ต้องการ
geojson = [mapping(clip_polygon)]

with rasterio.open('data/dem_sample.tif') as src:
    # Clip!
    clipped, clipped_transform = mask(
        src, 
        geojson, 
        crop=True,       # ตัดขนาดภาพให้พอดี
        nodata=-9999,    # กำหนดค่านอกขอบเขต
        filled=True
    )
    
    # เตรียม profile สำหรับไฟล์ใหม่
    out_profile = src.profile.copy()
    out_profile.update(
        height=clipped.shape[1],
        width=clipped.shape[2],
        transform=clipped_transform
    )

# เขียนผลลัพธ์
with rasterio.open('data/dem_clipped.tif', 'w', **out_profile) as dst:
    dst.write(clipped)

# แสดงผลเปรียบเทียบ
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

with rasterio.open('data/dem_sample.tif') as src:
    show(src, ax=axes[0], cmap='terrain', title='Original')
    
with rasterio.open('data/dem_clipped.tif') as src:
    show(src, ax=axes[1], cmap='terrain', title='Clipped')

plt.tight_layout()
plt.show()
print(f"✅ ตัดภาพเรียบร้อย: {clipped.shape[2]}×{clipped.shape[1]} pixels")

## 2.7 Raster Statistics & Zonal Statistics

In [ ]:
# สถิติพื้นฐาน
with rasterio.open('data/dem_sample.tif') as src:
    dem = src.read(1)
    
    # จัดการ NoData
    dem_masked = np.ma.masked_equal(dem, src.nodata)
    
    print("📊 สถิติ DEM")
    print(f"   Min       : {dem_masked.min():.1f} m")
    print(f"   Max       : {dem_masked.max():.1f} m")
    print(f"   Mean      : {dem_masked.mean():.1f} m")
    print(f"   Std       : {dem_masked.std():.1f} m")
    print(f"   Median    : {np.ma.median(dem_masked):.1f} m")

# Histogram
fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(dem.ravel(), bins=50, color='steelblue', edgecolor='white', alpha=0.8)
ax.set_xlabel('Elevation (m)')
ax.set_ylabel('Frequency')
ax.set_title('DEM Histogram')
ax.axvline(dem.mean(), color='red', linestyle='--', label=f'Mean: {dem.mean():.0f} m')
ax.legend()
plt.tight_layout()
plt.show()

## 2.8 การอ่าน Raster จาก URL (Cloud Optimized GeoTIFF)

Rasterio สามารถอ่าน COG จาก URL ได้โดยตรง ไม่ต้องดาวน์โหลดทั้งไฟล์

In [ ]:
# ตัวอย่างการอ่าน COG จาก URL
# (ใช้ข้อมูล Copernicus DEM จาก AWS Open Data)

# url = "https://copernicus-dem-30m.s3.amazonaws.com/..."
# with rasterio.open(url) as src:
#     # อ่านเฉพาะส่วนที่ต้องการ (ไม่ต้องโหลดทั้งไฟล์!)
#     window = from_bounds(98.5, 18.5, 99.0, 19.0, src.transform)
#     subset = src.read(1, window=window)

print("💡 COG ช่วยให้อ่านเฉพาะส่วนที่ต้องการจาก cloud ได้")
print("   ไม่ต้องดาวน์โหลดไฟล์ขนาดใหญ่ทั้งหมด")
print("   เหมาะกับ AWS S3, Google Cloud Storage, Azure Blob")

---
# Part 3: Xarray — จัดการ Raster แบบ N-dimensional

**Xarray** เป็น Library สำหรับจัดการข้อมูลหลายมิติ (N-dimensional) พร้อม labels

### ทำไมต้องใช้ Xarray?

| Rasterio | Xarray |
|----------|--------|
| อ่าน/เขียนไฟล์เร็ว | จัดการ dimensions (time, band, x, y) ง่าย |
| ได้ numpy array | ได้ labeled array (มีชื่อ coordinates) |
| เหมาะงาน I/O | เหมาะวิเคราะห์ time-series |
| ต้องจัดการ CRS เอง | ใช้ร่วมกับ rioxarray เพิ่ม CRS |
| ประมวลผลใน memory | รองรับ Dask (lazy loading) |

### โครงสร้างข้อมูล Xarray

```
Dataset (เหมือน dictionary ของ DataArray)
├── DataArray: temperature  (time, lat, lon)
├── DataArray: precipitation (time, lat, lon)
├── Coordinates:
│   ├── time: [2024-01, 2024-02, ...]
│   ├── lat:  [20.0, 19.9, 19.8, ...]
│   └── lon:  [98.0, 98.1, 98.2, ...]
└── Attributes: {title, source, crs, ...}
```

## 3.1 เปิดไฟล์ NetCDF

In [ ]:
import xarray as xr

# เปิดไฟล์ NetCDF
ds = xr.open_dataset('data/temperature_monthly.nc')

# สำรวจ Dataset
print("=" * 60)
print("📋 DATASET OVERVIEW")
print("=" * 60)
print(ds)
print("\n" + "=" * 60)
print("📐 DIMENSIONS")
print("=" * 60)
print(ds.dims)
print("\n" + "=" * 60)
print("📌 COORDINATES")
print("=" * 60)
print(ds.coords)
print("\n" + "=" * 60)
print("📦 DATA VARIABLES")
print("=" * 60)
for var in ds.data_vars:
    print(f"  {var}: {ds[var].dims} → shape {ds[var].shape}")

## 3.2 DataArray — หัวใจของ Xarray

In [ ]:
# ดึง DataArray จาก Dataset
temp = ds['temperature']

print("=" * 50)
print("🌡️ Temperature DataArray")
print("=" * 50)
print(f"Dimensions : {temp.dims}")
print(f"Shape      : {temp.shape}")
print(f"Dtype      : {temp.dtype}")
print(f"Units      : {temp.attrs.get('units', 'N/A')}")
print(f"\nค่าสถิติ:")
print(f"  Min  : {float(temp.min()):.1f}°C")
print(f"  Max  : {float(temp.max()):.1f}°C")
print(f"  Mean : {float(temp.mean()):.1f}°C")

## 3.3 การเลือกข้อมูล (Selection/Indexing)

In [ ]:
# ===== การเลือกข้อมูลใน Xarray =====

# 1. เลือกด้วย label (แนะนำ!)
jan = temp.sel(time='2024-01')  # เดือนมกราคม
print(f"มกราคม: shape = {jan.shape}")

# 2. เลือกช่วงเวลา
q1 = temp.sel(time=slice('2024-01', '2024-03'))  # ม.ค. - มี.ค.
print(f"Q1: shape = {q1.shape}")

# 3. เลือกตำแหน่ง (nearest neighbor)
point = temp.sel(latitude=19.0, longitude=99.0, method='nearest')
print(f"\nอุณหภูมิที่ lat=19, lon=99 (ทั้งปี): shape = {point.shape}")
print(f"ค่า: {point.values}")

# 4. เลือกด้วย index
first_month = temp.isel(time=0)  # เดือนแรก
print(f"\nเดือนแรก: shape = {first_month.shape}")

# 5. เลือกด้วยเงื่อนไข (Boolean indexing)
hot_areas = temp.where(temp > 30)  # เฉพาะพื้นที่ > 30°C
print(f"\nพื้นที่ร้อน > 30°C: {int(hot_areas.count())} pixels")

## 3.4 การคำนวณกับ Xarray

In [ ]:
# ===== การคำนวณสถิติตามมิติต่างๆ =====

# ค่าเฉลี่ยรายปี (เฉลี่ยตามมิติ time)
annual_mean = temp.mean(dim='time')
print(f"ค่าเฉลี่ยรายปี: shape = {annual_mean.shape}")

# ค่าเฉลี่ยเชิงพื้นที่ (เฉลี่ยตามมิติ lat, lon → ได้ time-series)
spatial_mean = temp.mean(dim=['latitude', 'longitude'])
print(f"ค่าเฉลี่ยเชิงพื้นที่: shape = {spatial_mean.shape}")

# ค่า Max, Min ตามเวลา
max_temp = temp.max(dim='time')
min_temp = temp.min(dim='time')
temp_range = max_temp - min_temp
print(f"ช่วงอุณหภูมิ (Max-Min): {float(temp_range.mean()):.1f}°C")

In [ ]:
# แสดงผล: อุณหภูมิเฉลี่ยรายเดือน (เชิงพื้นที่)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Time-series plot
spatial_mean.plot(ax=axes[0], marker='o', linewidth=2, color='orangered')
axes[0].set_title('อุณหภูมิเฉลี่ยเชิงพื้นที่รายเดือน')
axes[0].set_ylabel('Temperature (°C)')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim(15, 40)

# แผนที่อุณหภูมิเฉลี่ยรายปี
annual_mean.plot(ax=axes[1], cmap='RdYlBu_r', 
                 cbar_kwargs={'label': 'Temperature (°C)'})
axes[1].set_title('อุณหภูมิเฉลี่ยรายปี')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')

plt.tight_layout()
plt.show()

In [ ]:
# แสดงอุณหภูมิรายเดือน (facet plot)
fig = temp.plot(
    col='time',      # แบ่งคอลัมน์ตามเวลา
    col_wrap=4,       # 4 คอลัมน์ต่อแถว
    cmap='RdYlBu_r',
    figsize=(16, 10),
    cbar_kwargs={'label': 'Temperature (°C)', 'shrink': 0.8}
)

plt.suptitle('อุณหภูมิรายเดือน - ภาคเหนือ (จำลอง)', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

## 3.5 GroupBy และ Resample

In [ ]:
# ===== GroupBy: จัดกลุ่มตามเงื่อนไข =====

# แบ่งตามฤดูกาล
seasonal_mean = temp.groupby('time.season').mean(dim='time')
print("อุณหภูมิเฉลี่ยตามฤดูกาล:")
for season in seasonal_mean.season.values:
    mean_val = float(seasonal_mean.sel(season=season).mean())
    print(f"  {season}: {mean_val:.1f}°C")

# แบ่งตามเดือน
monthly_clim = temp.groupby('time.month').mean(dim='time')
print(f"\nClimatology shape: {monthly_clim.shape}")

In [ ]:
# แสดงผลตามฤดูกาล
fig, axes = plt.subplots(1, 4, figsize=(20, 4))
seasons = ['DJF', 'MAM', 'JJA', 'SON']
season_names = ['หนาว (DJF)', 'ร้อน (MAM)', 'ฝน (JJA)', 'ปลายฝน (SON)']

for i, (season, name) in enumerate(zip(seasons, season_names)):
    seasonal_mean.sel(season=season).plot(
        ax=axes[i], cmap='RdYlBu_r', 
        vmin=18, vmax=35,
        add_colorbar=True if i == 3 else False
    )
    axes[i].set_title(name)

plt.suptitle('อุณหภูมิเฉลี่ยตามฤดูกาล', fontsize=14)
plt.tight_layout()
plt.show()

## 3.6 Lazy Loading ด้วย Dask (สำหรับข้อมูลขนาดใหญ่)

In [ ]:
# ===== Dask Integration =====
# เมื่อข้อมูลใหญ่เกินไปสำหรับ RAM ให้ใช้ Dask

# เปิดไฟล์แบบ lazy (ไม่โหลดข้อมูลจริงจนกว่าจะใช้)
ds_lazy = xr.open_dataset('data/temperature_monthly.nc', 
                           chunks={'time': 3, 'latitude': 25, 'longitude': 25})

print("=== Lazy Loading ===")
print(f"Type: {type(ds_lazy['temperature'].data)}")  # dask array
print(f"Chunks: {ds_lazy['temperature'].chunks}")
print(f"\n💡 ข้อมูลยังไม่ถูกโหลดเข้า memory!")

# คำนวณ (ยังเป็น lazy)
result_lazy = ds_lazy['temperature'].mean(dim='time')
print(f"\nResult (lazy): {result_lazy}")

# .compute() เพื่อรันจริง
result = result_lazy.compute()
print(f"\nResult (computed): shape = {result.shape}")
print(f"Mean: {float(result.mean()):.1f}°C")

In [ ]:
# เปิดหลายไฟล์พร้อมกัน (ตัวอย่าง pattern)
# ds_multi = xr.open_mfdataset('data/temp_*.nc', 
#                               chunks={'time': 12},
#                               combine='by_coords')

print("💡 open_mfdataset() ใช้สำหรับ:")
print("   - เปิดหลายไฟล์ NetCDF ที่แบ่งตามเวลา")
print("   - เช่น temp_2020.nc, temp_2021.nc, temp_2022.nc")
print("   - Xarray จะรวมเป็น Dataset เดียวอัตโนมัติ")
print("   - ใช้ Dask chunks เพื่อจัดการ memory")

---
# Part 4: Rioxarray — สะพานเชื่อม Rasterio + Xarray

**Rioxarray** เพิ่มความสามารถทาง geospatial ให้กับ Xarray:
- อ่าน/เขียน GeoTIFF ผ่าน Xarray
- จัดการ CRS (Coordinate Reference System)
- Reproject, Clip, Resampling

```python
import rioxarray  # แค่ import ก็เพิ่ม .rio accessor ให้ Xarray
```

## 4.1 เปิดไฟล์ GeoTIFF ด้วย Rioxarray

In [ ]:
import rioxarray

# เปิดไฟล์ GeoTIFF → Xarray DataArray
da = rioxarray.open_rasterio('data/dem_sample.tif')

print("=" * 50)
print("📋 DataArray จาก GeoTIFF")
print("=" * 50)
print(f"Dimensions : {da.dims}")
print(f"Shape      : {da.shape}")
print(f"CRS        : {da.rio.crs}")
print(f"Resolution : {da.rio.resolution()}")
print(f"Bounds     : {da.rio.bounds()}")
print(f"NoData     : {da.rio.nodata}")
print(f"Transform  : {da.rio.transform()}")
print(f"\nCoordinates:")
print(f"  x: {float(da.x.min()):.2f} to {float(da.x.max()):.2f}")
print(f"  y: {float(da.y.min()):.2f} to {float(da.y.max()):.2f}")

In [ ]:
# เปิดไฟล์ Multispectral
ms = rioxarray.open_rasterio('data/multispectral_sample.tif')

print(f"Shape: {ms.shape}")  # (bands, y, x)
print(f"Band names: {list(ms.band.values)}")

# เลือก band
red = ms.sel(band=3)
nir = ms.sel(band=4)

# คำนวณ NDVI (ง่ายมากกับ Xarray!)
ndvi = (nir - red) / (nir + red)
ndvi.name = 'NDVI'

# Plot
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

red.plot(ax=axes[0], cmap='Reds')
axes[0].set_title('Red Band')

nir.plot(ax=axes[1], cmap='YlGn')
axes[1].set_title('NIR Band')

ndvi.plot(ax=axes[2], cmap='RdYlGn', vmin=-0.2, vmax=0.8)
axes[2].set_title('NDVI')

plt.tight_layout()
plt.show()

## 4.2 Reproject ด้วย Rioxarray

In [ ]:
# Reproject (ง่ายกว่า Rasterio มาก!)
da_utm = da.rio.reproject('EPSG:32647')  # บรรทัดเดียว!

print(f"CRS เดิม  : {da.rio.crs}")
print(f"CRS ใหม่  : {da_utm.rio.crs}")
print(f"Shape เดิม : {da.shape}")
print(f"Shape ใหม่ : {da_utm.shape}")
print(f"Resolution : {da_utm.rio.resolution()}")

# Reproject พร้อมกำหนด resolution
da_utm_1000m = da.rio.reproject(
    'EPSG:32647',
    resolution=1000,  # 1000 เมตร
    resampling=rioxarray.rasterio.enums.Resampling.bilinear
)
print(f"\nResample 1000m: shape = {da_utm_1000m.shape}")

## 4.3 Clip ด้วย Rioxarray

In [ ]:
from shapely.geometry import box, Polygon
import geopandas as gpd

# วิธีที่ 1: Clip ด้วย bounding box
da_bbox = da.rio.clip_box(minx=98.5, miny=18.5, maxx=99.5, maxy=19.5)
print(f"Clip by bbox: {da_bbox.shape}")

# วิธีที่ 2: Clip ด้วย polygon (geometry)
polygon = box(98.5, 18.5, 99.5, 19.5)
da_clipped = da.rio.clip([polygon], crs='EPSG:4326')
print(f"Clip by polygon: {da_clipped.shape}")

# วิธีที่ 3: Clip ด้วย GeoDataFrame (เช่น shapefile)
# gdf = gpd.read_file('boundary.shp')
# da_clipped = da.rio.clip(gdf.geometry, gdf.crs)

# แสดงผล
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
da.sel(band=1).plot(ax=axes[0], cmap='terrain')
axes[0].set_title('Original')
da_clipped.sel(band=1).plot(ax=axes[1], cmap='terrain')
axes[1].set_title('Clipped')
plt.tight_layout()
plt.show()

## 4.4 เขียนไฟล์ด้วย Rioxarray

In [ ]:
# เขียน DataArray เป็น GeoTIFF
ndvi.rio.set_crs('EPSG:4326')  # กำหนด CRS
ndvi.rio.set_nodata(-9999)       # กำหนด NoData

ndvi.rio.to_raster(
    'data/ndvi_rioxarray.tif',
    driver='GTiff',
    dtype='float32',
    compress='lzw'
)

print("✅ เขียนไฟล์ data/ndvi_rioxarray.tif เรียบร้อย")

# ตรวจสอบ
check = rioxarray.open_rasterio('data/ndvi_rioxarray.tif')
print(f"   CRS  : {check.rio.crs}")
print(f"   Shape: {check.shape}")

## 4.5 เพิ่ม CRS ให้ Xarray Dataset

In [ ]:
# เปิด NetCDF ที่ไม่มี CRS
ds = xr.open_dataset('data/temperature_monthly.nc')

# เพิ่ม spatial dimensions
temp = ds['temperature']
temp = temp.rename({'latitude': 'y', 'longitude': 'x'})

# กำหนด CRS
temp.rio.write_crs('EPSG:4326', inplace=True)
temp.rio.set_spatial_dims(x_dim='x', y_dim='y', inplace=True)

print(f"CRS: {temp.rio.crs}")
print(f"Bounds: {temp.rio.bounds()}")

# ตอนนี้ทำ geo-operations ได้แล้ว
jan_clip = temp.sel(time='2024-01').rio.clip_box(
    minx=98.5, miny=18.5, maxx=99.5, maxy=19.5
)
print(f"Clipped January: shape = {jan_clip.shape}")

---
# Part 5: XEE — เชื่อมต่อ Google Earth Engine กับ Xarray

**XEE (Xarray Earth Engine)** ช่วยให้ดึงข้อมูลจาก Google Earth Engine มาเป็น Xarray Dataset/DataArray ได้โดยตรง

### ข้อดีของ XEE

| แบบเดิม (GEE Python API) | ใช้ XEE |
|--------------------------|--------|
| คำนวณบน server | ดึงมาคำนวณใน Python |
| Export เป็นไฟล์ → โหลด → เปิด | ได้ Xarray โดยตรง |
| ใช้ GEE syntax | ใช้ Xarray syntax ที่คุ้นเคย |
| จำกัด visualization | ใช้ matplotlib, plotly ได้เต็มที่ |

### ⚠️ ข้อจำกัด
- ต้องมี Google Earth Engine account
- ต้อง authenticate ก่อนใช้งาน
- ข้อมูลถูก download มาที่เครื่อง (อาจช้าสำหรับข้อมูลใหญ่)

## 5.1 การติดตั้งและ Authentication

In [ ]:
# ติดตั้ง (ถ้ายังไม่ได้ติดตั้ง)
# !pip install xee earthengine-api

import ee
import xarray as xr

# === Authentication ===
# ครั้งแรกจะเปิด browser ให้ login
# ee.Authenticate()  # รันครั้งเดียว

# Initialize
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
    print("✅ Google Earth Engine พร้อมใช้งาน!")
except Exception as e:
    print(f"⚠️ ต้อง Authenticate ก่อน: ee.Authenticate()")
    print(f"   Error: {e}")

## 5.2 ดึงข้อมูล ImageCollection → Xarray

In [ ]:
# ===== ตัวอย่าง: ดึงข้อมูล Sentinel-2 =====
# (ต้อง authenticate สำเร็จก่อน)

try:
    # กำหนดพื้นที่สนใจ (เชียงใหม่)
    roi = ee.Geometry.Rectangle([98.8, 18.6, 99.2, 18.9])
    
    # เลือก Image Collection
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(roi)
          .filterDate('2024-01-01', '2024-03-31')
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
          .select(['B2', 'B3', 'B4', 'B8'])  # Blue, Green, Red, NIR
    )
    
    print(f"จำนวนภาพ: {s2.size().getInfo()}")
    
    # ===== แปลงเป็น Xarray ด้วย XEE =====
    ds = xr.open_dataset(
        s2,
        engine='ee',           # ⭐ ใช้ XEE engine
        crs='EPSG:32647',      # ระบบพิกัดที่ต้องการ
        scale=100,             # ความละเอียด (เมตร) — ใช้ 100m เพื่อความเร็ว
        geometry=roi,          # พื้นที่สนใจ
    )
    
    print("\n✅ ดึงข้อมูลสำเร็จ!")
    print(ds)
    
except Exception as e:
    print(f"⚠️ ไม่สามารถเชื่อมต่อ GEE ได้")
    print(f"   ให้รัน ee.Authenticate() ก่อน")
    print(f"   Error: {e}")

## 5.3 ตัวอย่าง: ดึงข้อมูล MODIS LST

In [ ]:
# ===== ดึงข้อมูล MODIS Land Surface Temperature =====

try:
    roi = ee.Geometry.Rectangle([98.5, 18.5, 99.5, 19.5])
    
    modis_lst = (ee.ImageCollection('MODIS/061/MOD11A2')
                .filterBounds(roi)
                .filterDate('2024-01-01', '2024-12-31')
                .select(['LST_Day_1km', 'LST_Night_1km'])
    )
    
    ds_lst = xr.open_dataset(
        modis_lst,
        engine='ee',
        crs='EPSG:4326',
        scale=1000,       # MODIS 1km
        geometry=roi,
    )
    
    # แปลงหน่วย (MODIS LST: Scale factor = 0.02, offset = 0)
    # หน่วย: Kelvin → Celsius
    ds_lst['LST_Day_C'] = ds_lst['LST_Day_1km'] * 0.02 - 273.15
    ds_lst['LST_Night_C'] = ds_lst['LST_Night_1km'] * 0.02 - 273.15
    
    print("✅ ดึงข้อมูล MODIS LST สำเร็จ!")
    print(ds_lst)
    
    # วิเคราะห์ด้วย Xarray ตามปกติ
    monthly_lst = ds_lst['LST_Day_C'].resample(time='MS').mean()
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 4))
    monthly_lst.mean(dim=['lat', 'lon']).plot(ax=ax, marker='o')
    ax.set_title('MODIS Daytime LST - ภาคเหนือ 2024')
    ax.set_ylabel('Temperature (°C)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"⚠️ ต้อง authenticate GEE ก่อน")
    print(f"   Error: {e}")

## 5.4 ตัวอย่าง: ดึง DEM จาก GEE

In [ ]:
# ===== ดึง SRTM DEM =====

try:
    roi = ee.Geometry.Rectangle([98.5, 18.5, 99.5, 19.5])
    
    # SRTM เป็น ee.Image (ไม่ใช่ ImageCollection)
    srtm = ee.Image('USGS/SRTMGL1_003').select('elevation')
    
    # สำหรับ ee.Image ต้องแปลงเป็น ImageCollection ก่อน
    srtm_ic = ee.ImageCollection([srtm])
    
    ds_dem = xr.open_dataset(
        srtm_ic,
        engine='ee',
        crs='EPSG:4326',
        scale=90,        # SRTM ~90m
        geometry=roi,
    )
    
    print("✅ ดึง SRTM DEM สำเร็จ!")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 8))
    ds_dem['elevation'].isel(time=0).plot(
        ax=ax, cmap='terrain',
        cbar_kwargs={'label': 'Elevation (m)'}
    )
    ax.set_title('SRTM DEM - ภาคเหนือ')
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"⚠️ ต้อง authenticate GEE ก่อน")
    print(f"   Error: {e}")

## 5.5 บันทึกข้อมูลจาก XEE เป็นไฟล์

In [ ]:
# ===== บันทึกข้อมูลที่ดึงจาก GEE =====

# บันทึกเป็น NetCDF (เหมาะกับ time-series)
# ds_lst.to_netcdf('data/modis_lst_2024.nc')

# บันทึกเป็น GeoTIFF (เหมาะกับ single image)
# import rioxarray
# ds_dem['elevation'].isel(time=0).rio.to_raster('data/srtm_dem.tif')

# บันทึกเป็น Zarr (เหมาะกับ cloud + ข้อมูลใหญ่)
# ds_lst.to_zarr('data/modis_lst_2024.zarr')

print("💡 รูปแบบไฟล์ที่แนะนำ:")
print("   📄 GeoTIFF → ภาพเดี่ยว, ใช้กับ GIS ทั่วไป")
print("   📄 NetCDF  → Time-series, หลายตัวแปร")
print("   📄 Zarr    → Cloud storage, ข้อมูลขนาดใหญ่มาก")

---
# Part 6: Workshop — ประยุกต์ใช้งานจริง

## 🏋️ Workshop 1: วิเคราะห์ NDVI Time-series

In [ ]:
# ===== Workshop 1: สร้าง NDVI Time-series จำลอง =====

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# สร้างข้อมูลจำลอง NDVI รายเดือน (มีรูปแบบตามฤดูกาล)
times = pd.date_range('2020-01-01', '2023-12-31', freq='MS')
lats = np.linspace(19.5, 18.5, 30)
lons = np.linspace(98.5, 99.5, 30)

# NDVI จำลอง (สูงในฤดูฝน ต่ำในฤดูแล้ง)
ndvi_data = np.zeros((len(times), 30, 30), dtype=np.float32)
for t, time in enumerate(times):
    month = time.month
    # Seasonal pattern: สูงสุดเดือน ก.ย. (หลังฤดูฝน)
    seasonal = 0.5 + 0.2 * np.sin(2*np.pi*(month-3)/12)
    # Spatial variation
    LON, LAT = np.meshgrid(lons, lats)
    spatial = 0.1 * np.sin(LON * 5) * np.cos(LAT * 5)
    ndvi_data[t] = seasonal + spatial + np.random.normal(0, 0.05, (30, 30))

ndvi_data = np.clip(ndvi_data, -0.2, 0.9)

# สร้าง Dataset
ds_ndvi = xr.Dataset({
    'ndvi': (['time', 'latitude', 'longitude'], ndvi_data,
             {'long_name': 'Normalized Difference Vegetation Index'})
}, coords={
    'time': times,
    'latitude': lats,
    'longitude': lons
})

print("✅ สร้างข้อมูล NDVI Time-series สำเร็จ")
print(ds_ndvi)

In [ ]:
# วิเคราะห์ NDVI Time-series

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. NDVI เฉลี่ยเชิงพื้นที่ตลอดช่วงเวลา
ndvi_spatial_mean = ds_ndvi['ndvi'].mean(dim=['latitude', 'longitude'])
ndvi_spatial_mean.plot(ax=axes[0, 0], linewidth=1.5, color='green')
axes[0, 0].set_title('NDVI เฉลี่ยเชิงพื้นที่รายเดือน')
axes[0, 0].set_ylabel('NDVI')
axes[0, 0].grid(True, alpha=0.3)

# 2. NDVI เฉลี่ยรายปี (เปรียบเทียบปีต่อปี)
for year in [2020, 2021, 2022, 2023]:
    yearly = ds_ndvi['ndvi'].sel(time=str(year)).mean(dim=['latitude', 'longitude'])
    axes[0, 1].plot(range(1, len(yearly)+1), yearly.values, 
                    marker='o', label=str(year), linewidth=1.5)
axes[0, 1].set_title('เปรียบเทียบ NDVI รายปี')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('NDVI')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. แผนที่ NDVI เฉลี่ย
ds_ndvi['ndvi'].mean(dim='time').plot(
    ax=axes[1, 0], cmap='YlGn',
    cbar_kwargs={'label': 'NDVI'}
)
axes[1, 0].set_title('NDVI เฉลี่ย 4 ปี')

# 4. NDVI Trend (ค่าเฉลี่ยรายปี)
annual_ndvi = ds_ndvi['ndvi'].groupby('time.year').mean()
trend = annual_ndvi.mean(dim=['latitude', 'longitude'])
trend.plot(ax=axes[1, 1], marker='s', linewidth=2, color='darkgreen', markersize=10)
axes[1, 1].set_title('NDVI Trend (ค่าเฉลี่ยรายปี)')
axes[1, 1].set_ylabel('NDVI')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Workshop 1: การวิเคราะห์ NDVI Time-series', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 🏋️ Workshop 2: เปรียบเทียบ DEM จาก 2 แหล่ง

In [ ]:
# ===== Workshop 2: สร้าง DEM 2 ชุดที่มี resolution ต่างกัน =====

import rasterio
import rioxarray
from rasterio.transform import from_bounds
import numpy as np

# DEM ชุดที่ 1: High-res (100x100)
nrows1, ncols1 = 100, 100
x1 = np.linspace(0, 4*np.pi, ncols1)
y1 = np.linspace(0, 4*np.pi, nrows1)
X1, Y1 = np.meshgrid(x1, y1)
dem1 = (500 + 200*np.sin(X1)*np.cos(Y1) + np.random.normal(0, 5, (nrows1, ncols1))).astype(np.float32)

transform1 = from_bounds(98.5, 18.5, 99.5, 19.5, ncols1, nrows1)
with rasterio.open('data/dem_highres.tif', 'w', driver='GTiff',
                    height=nrows1, width=ncols1, count=1, dtype='float32',
                    crs='EPSG:4326', transform=transform1) as dst:
    dst.write(dem1, 1)

# DEM ชุดที่ 2: Low-res (25x25) + noise มากกว่า
nrows2, ncols2 = 25, 25
x2 = np.linspace(0, 4*np.pi, ncols2)
y2 = np.linspace(0, 4*np.pi, nrows2)
X2, Y2 = np.meshgrid(x2, y2)
dem2 = (500 + 200*np.sin(X2)*np.cos(Y2) + np.random.normal(0, 20, (nrows2, ncols2))).astype(np.float32)

transform2 = from_bounds(98.5, 18.5, 99.5, 19.5, ncols2, nrows2)
with rasterio.open('data/dem_lowres.tif', 'w', driver='GTiff',
                    height=nrows2, width=ncols2, count=1, dtype='float32',
                    crs='EPSG:4326', transform=transform2) as dst:
    dst.write(dem2, 1)

print("✅ สร้าง DEM 2 ชุดสำเร็จ")
print(f"   High-res: {ncols1}×{nrows1} pixels")
print(f"   Low-res:  {ncols2}×{nrows2} pixels")

In [ ]:
# เปรียบเทียบ DEM 2 ชุด

# เปิดด้วย rioxarray
dem_hi = rioxarray.open_rasterio('data/dem_highres.tif').sel(band=1)
dem_lo = rioxarray.open_rasterio('data/dem_lowres.tif').sel(band=1)

print(f"High-res shape: {dem_hi.shape}, resolution: {dem_hi.rio.resolution()}")
print(f"Low-res shape:  {dem_lo.shape}, resolution: {dem_lo.rio.resolution()}")

# Resample low-res ให้ตรงกับ high-res (ใช้ reproject_match)
dem_lo_resampled = dem_lo.rio.reproject_match(dem_hi)
print(f"\nLow-res (resampled): {dem_lo_resampled.shape}")

# คำนวณความแตกต่าง
diff = dem_hi - dem_lo_resampled

# แสดงผล
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

dem_hi.plot(ax=axes[0], cmap='terrain', cbar_kwargs={'label': 'm'})
axes[0].set_title(f'High-res ({dem_hi.shape[0]}×{dem_hi.shape[1]})')

dem_lo_resampled.plot(ax=axes[1], cmap='terrain', cbar_kwargs={'label': 'm'})
axes[1].set_title(f'Low-res (resampled to {dem_lo_resampled.shape[0]}×{dem_lo_resampled.shape[1]})')

diff.plot(ax=axes[2], cmap='RdBu_r', cbar_kwargs={'label': 'Difference (m)'})
axes[2].set_title(f'Difference (RMSE: {float(np.sqrt((diff**2).mean())):.1f} m)')

plt.suptitle('Workshop 2: เปรียบเทียบ DEM', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 🏋️ Workshop 3: สร้าง Raster Pipeline สมบูรณ์

In [ ]:
# ===== Workshop 3: Pipeline สมบูรณ์ =====
# Input: Multispectral image
# Process: Clip → Calculate indices → Classify → Export

import rioxarray
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.geometry import box

# Step 1: โหลดข้อมูล
print("Step 1: โหลดข้อมูล...")
ms = rioxarray.open_rasterio('data/multispectral_sample.tif')
print(f"  Original shape: {ms.shape}")

# Step 2: Clip พื้นที่สนใจ
print("Step 2: Clip...")
aoi = box(98.5, 18.5, 99.5, 19.5)
ms_clip = ms.rio.clip([aoi], crs='EPSG:4326')
print(f"  Clipped shape: {ms_clip.shape}")

# Step 3: คำนวณ Vegetation Indices
print("Step 3: คำนวณ Indices...")
red = ms_clip.sel(band=3).astype(float)
nir = ms_clip.sel(band=4).astype(float)
green = ms_clip.sel(band=2).astype(float)

# NDVI
ndvi = (nir - red) / (nir + red + 1e-10)
ndvi.name = 'NDVI'

# SAVI (Soil Adjusted Vegetation Index)
L = 0.5  # soil brightness correction factor
savi = ((nir - red) / (nir + red + L)) * (1 + L)
savi.name = 'SAVI'

# EVI (Enhanced Vegetation Index) - simplified
blue = ms_clip.sel(band=1).astype(float)
evi = 2.5 * (nir - red) / (nir + 6*red - 7.5*blue + 1)
evi.name = 'EVI'

print(f"  NDVI range: {float(ndvi.min()):.3f} to {float(ndvi.max()):.3f}")
print(f"  SAVI range: {float(savi.min()):.3f} to {float(savi.max()):.3f}")

# Step 4: จำแนกประเภท (Classification)
print("Step 4: จำแนกประเภท...")
# กำหนดเกณฑ์
land_class = ndvi.copy()
land_class.values = np.where(ndvi.values > 0.5, 3,      # ป่าทึบ
                    np.where(ndvi.values > 0.3, 2,       # ป่าโปร่ง/เกษตร
                    np.where(ndvi.values > 0.1, 1, 0)))  # ดิน/เมือง, น้ำ
land_class.name = 'Land Classification'

class_names = {0: 'น้ำ/เงา', 1: 'ดิน/เมือง', 2: 'เกษตร/ป่าโปร่ง', 3: 'ป่าทึบ'}
for cls, name in class_names.items():
    count = int((land_class == cls).sum())
    pct = count / land_class.size * 100
    print(f"  Class {cls} ({name}): {count} pixels ({pct:.1f}%)")

print("\n✅ Pipeline สมบูรณ์!")

In [ ]:
# แสดงผลลัพธ์ Pipeline

fig, axes = plt.subplots(2, 3, figsize=(18, 11))

# Row 1: Indices
ndvi.plot(ax=axes[0, 0], cmap='RdYlGn', vmin=-0.2, vmax=0.8,
          cbar_kwargs={'label': 'NDVI'})
axes[0, 0].set_title('NDVI')

savi.plot(ax=axes[0, 1], cmap='RdYlGn', vmin=-0.2, vmax=0.8,
          cbar_kwargs={'label': 'SAVI'})
axes[0, 1].set_title('SAVI')

evi.plot(ax=axes[0, 2], cmap='RdYlGn', vmin=-0.5, vmax=1.0,
         cbar_kwargs={'label': 'EVI'})
axes[0, 2].set_title('EVI')

# Row 2: Classification & Histogram
colors = ['#2196F3', '#FFC107', '#8BC34A', '#1B5E20']
cmap_class = ListedColormap(colors)
im = axes[1, 0].imshow(land_class.values, cmap=cmap_class, vmin=-0.5, vmax=3.5,
                        extent=[98.5, 99.5, 18.5, 19.5])
cbar = plt.colorbar(im, ax=axes[1, 0], ticks=[0, 1, 2, 3])
cbar.ax.set_yticklabels(['น้ำ', 'ดิน/เมือง', 'เกษตร', 'ป่าทึบ'])
axes[1, 0].set_title('Land Classification')

# NDVI Histogram
axes[1, 1].hist(ndvi.values.ravel(), bins=50, color='green', alpha=0.7, edgecolor='white')
axes[1, 1].axvline(0.1, color='red', linestyle='--', label='Threshold 0.1')
axes[1, 1].axvline(0.3, color='orange', linestyle='--', label='Threshold 0.3')
axes[1, 1].axvline(0.5, color='darkgreen', linestyle='--', label='Threshold 0.5')
axes[1, 1].set_title('NDVI Distribution')
axes[1, 1].set_xlabel('NDVI')
axes[1, 1].legend(fontsize=8)

# NDVI vs SAVI scatter
sample_idx = np.random.choice(ndvi.size, size=min(5000, ndvi.size), replace=False)
axes[1, 2].scatter(ndvi.values.ravel()[sample_idx], 
                   savi.values.ravel()[sample_idx],
                   s=1, alpha=0.3, color='green')
axes[1, 2].plot([-0.2, 0.8], [-0.2, 0.8], 'r--', label='1:1 line')
axes[1, 2].set_xlabel('NDVI')
axes[1, 2].set_ylabel('SAVI')
axes[1, 2].set_title('NDVI vs SAVI')
axes[1, 2].legend()

plt.suptitle('Workshop 3: Raster Pipeline สมบูรณ์', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Step 5: Export ผลลัพธ์
print("Step 5: Export ผลลัพธ์...")

# บันทึก NDVI
ndvi_out = ndvi.copy()
ndvi_out.rio.write_crs('EPSG:4326', inplace=True)
ndvi_out.rio.to_raster('data/output_ndvi.tif', dtype='float32', compress='lzw')
print("  ✅ data/output_ndvi.tif")

# บันทึก Classification
class_out = land_class.copy()
class_out.rio.write_crs('EPSG:4326', inplace=True)
class_out.rio.to_raster('data/output_classification.tif', dtype='uint8', compress='lzw')
print("  ✅ data/output_classification.tif")

print("\n🎉 เสร็จสิ้น Workshop 3!")

---
# 📋 สรุป: เลือกใช้ Library ไหนดี?

```
คำถาม                              → Library ที่แนะนำ
─────────────────────────────────────────────────────
อ่าน/เขียน GeoTIFF ง่ายๆ           → Rasterio
ดูข้อมูล metadata ของไฟล์          → Rasterio
ตัดภาพด้วย shapefile               → Rasterio (mask) หรือ Rioxarray (.rio.clip)
เปลี่ยน CRS                        → Rasterio (reproject) หรือ Rioxarray (.rio.reproject)
วิเคราะห์ time-series              → Xarray
ข้อมูล NetCDF/HDF5                 → Xarray
ข้อมูลใหญ่เกินไปสำหรับ RAM         → Xarray + Dask (chunks)
ต้องการ CRS awareness + Xarray     → Rioxarray
ดึงข้อมูลจาก Google Earth Engine   → XEE
ใช้ GEE data กับ Xarray workflow    → XEE + Rioxarray
```

### Workflow แนะนำ

```
1. GEE data    → XEE → Xarray → Rioxarray → GeoTIFF
2. GeoTIFF     → Rioxarray → Xarray → วิเคราะห์ → Export
3. NetCDF      → Xarray → Rioxarray → Clip/Reproject → Export
4. ไฟล์เดี่ยว   → Rasterio → NumPy → วิเคราะห์ → Rasterio (เขียน)
```

---
# 📚 แหล่งเรียนรู้เพิ่มเติม

| แหล่ง | URL |
|-------|-----|
| Rasterio Documentation | https://rasterio.readthedocs.io |
| Xarray Documentation | https://docs.xarray.dev |
| Rioxarray Documentation | https://corteva.github.io/rioxarray |
| XEE GitHub | https://github.com/google/Xee |
| GEE Data Catalog | https://developers.google.com/earth-engine/datasets |
| Cloud-Native Geospatial | https://cloudnativegeo.org |
| Pangeo | https://pangeo.io |